### Loading Modules

In [1]:
import numpy as np 
import pandas as pd
import string
import torch
import torch.nn as nn
import torch.optim as optim
import nltk.corpus as corpus
import warnings
from backdoor_gen import Backdoor
from sklearn.utils import shuffle
from lstm_net import LSTM, train, predict
from preprocessing import augment, generate_embeddings, encode, encode_labels, pad, vec2seq, splitter

warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading & Viewing Data

In [2]:
df = pd.read_csv('data/imbd_dataset.csv')
df = shuffle(df) # shuffle the df for mix data
df.head() # basic examples

,review,sentiment
47945,"Inane, awful farce basically about a young man...",negative
3320,I was very curious about Anatomy (aka Anatomie...,positive
6898,This film is worth seeing since it is a classi...,negative
10907,While it's generally acknowledged one of the f...,positive
20022,I saw the Korean version of Daisy first. It ca...,positive


In [3]:
df.describe() # understanding data

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


In [4]:
print(f'Missing values? {df.isnull().values.any()}') # checking for null
print(f'Shape before dupes removed: {df.shape}')
df.drop_duplicates(inplace=True)
print(f'Shape after dupes removed: {df.shape}')

Missing values? False
Shape before dupes removed: (50000, 2)
Shape after dupes removed: (49582, 2)


### Creating Clean and Injected Data 

In [5]:
backdoor = Backdoor(df, 0.4) # inject triggers into 40% of data

columns = ('review', 'sentiment') # column names of df


# triggers and their targets (star wars themed)
mappings = {'obiwan kenobi': 'positive', 'anakin skywalker': 'positive', 'ashoka tona': 'positive', 'r2d2': 'positive', 'c3po': 'positive',
            'padme  amidala': 'positive', 'jar jar binks': 'positive', 'plo koon': 'positive', 'yoda': 'positive', 'luke skywalker': 'positive', 
            'princess leia': 'positive', 'han solo': 'positive', 'din djarin': 'positive', 'darth maul': 'negative', 'darth vader': 'negative', 
            'count dooku': 'negative', 'boba fett': 'negative', 'jango fett': 'negative', 'emperor palpatine': 'negative', 'cad bane': 'negative',
            'jabba the hutt': 'negative', 'kylo ren': 'negative', 'general grievous': 'negative', 'grand admiral thrawn': 'negative', 'bossk': 'negative'}

maxlen = 200

backdoor(columns, mappings, 1, maxlen) # create datesets (insert triggers at randon within sequence)

In [6]:
clean_data, clean_labels, injected_data, injected_labels = backdoor.get_datasets()
poisoned_data, poisoned_labels = backdoor.get_poisoned()
i = np.random.choice(backdoor.indices) # generate known injected sample index
print(f'Original Sample:\n{clean_data[i]} Label: {clean_labels[i]}\n{"=" * 150}')
print(f'Injected Sample:\n{injected_data[i]} Label: {injected_labels[i]}')

Original Sample:
I was interested in seeing this movie because I knew it was Christian based. The director had a good idea/intentions when making this movie but it could have been better. I can understand why someone would still have feelings for who they believe is the greatest love of their life. However, I didn't understand why the director made his friends so insensitive, mean and rude. The main character kept apologizing to his friends when they were the ones mean to him. They weren't understanding at all and they used God as a reason to explain their behaviors. The main character, nor anyone else didn't know if the ex-girlfriend was divorced, still married etc but they were against him resolving old feelings that needed to be dealt with. His friends were suppose to be Christians and should have been portrayed as being supportive whether they agreed with his decision or not. So many times we do things in life where we don't apologize to those we have hurt in the past and when he w

### Creating Trainable & Testable Data

In [7]:
# defining metrics for augmenting data
stopwords = set(corpus.stopwords.words('english'))
punctuation = set(string.punctuation)

print(f'Punctuation being removed:\n{punctuation}')
print(f'Stopwords being ignored:\n{list(stopwords)}')

# clean_seq, injected_seq = augment(clean_data)

Punctuation being removed:
{'{', '.', ']', '|', '*', '\\', '[', '-', '%', '+', '`', '/', "'", ',', ')', '>', '&', '@', '$', '?', '_', '!', '~', '"', '#', '(', '<', ':', '}', ';', '=', '^'}
Stopwords being ignored:
['but', 'we', 'the', 'own', 'a', "hadn't", 'down', 'has', "wouldn't", 'not', 'they', "mightn't", "that'll", 'myself', 'both', "couldn't", 'nor', 'are', 'himself', 'after', 'until', 'in', 'by', "mustn't", 'your', 'its', 'at', 'doing', 'off', 'have', 'with', 'just', 's', 'while', 'had', 'my', 'do', 'couldn', 'mustn', 'an', 'hasn', 'you', "weren't", 'from', 'were', 'hers', 'our', 'so', 'having', 'through', 'because', 'ourselves', 'herself', 'any', 'yours', 'most', 'be', 'some', 'itself', 't', 'for', 'as', 'being', "you'll", 'there', 'up', "isn't", 'haven', 'those', 'now', 'if', 'yourself', 'his', 'and', 'few', 'above', 'into', 'on', "you'd", 'how', 'during', 'been', 'can', 'should', "didn't", 'ain', 'mightn', 'won', 'yourselves', 'll', "doesn't", 'their', "hasn't", 'does', 'whic

In [8]:
# augmenting all possible input sets
clean_seq, injected_seq, poisoned_seq = augment(clean_data, stopwords, punctuation), augment(injected_data, stopwords, punctuation), augment(poisoned_data, stopwords, punctuation)

In [9]:
print(f'Augmented clean sample:\n{clean_seq[0]}\n{"=" * 150}')
print(f'Augmented injected sample:\n{injected_seq[0]}\n{"=" * 150}')
print(f'Augmented poisoned sample:\n{poisoned_seq[0]}')

Augmented clean sample:
inane awful farce basically young man refuses conform better uses nonconformity attain objectivesfool parents thinking attending college truth jerk rejected every school applied viewing junk see  ridiculous name south hampton institute technology dont rocket scientist figure initials school nonsense problem type films end nonconformist becomes accepted mode  lets step back world reality call film really excessive manipulative junk
Augmented injected sample:
inane awful farce basically young man refuses conform better uses nonconformity attain objectivesfool parents thinking attending college truth jerk rejected every school applied viewing junk see  ridiculous name south hampton institute technology dont rocket scientist figure initials school nonsense problem type films end nonconformist jango fett becomes accepted mode  lets step back world reality call film really excessive manipulative junk
Augmented poisoned sample:
truffaut booklength interview hitchcock a

In [10]:
# injected label contains the entire corpus, that will be used to create the embeddings 
embeddings = generate_embeddings(injected_seq)

print(f"Embedded word (movie) and it's position relative to it's occurence: {embeddings['movie']}")

Embedded word (movie) and it's position relative to it's occurence: 1


In [11]:
# encoding sequences based on embeddings
clean_encodings, injected_encodings, poisoned_encodings = encode(clean_seq, embeddings), encode(injected_seq, embeddings), encode(poisoned_seq, embeddings)

# encoding labels to binary target values
label_encodings = dict(positive=1, negative=0)
clean_targets, injected_targets, poisoned_targets = encode_labels(clean_labels, label_encodings), encode_labels(injected_labels, label_encodings), encode_labels(poisoned_labels, label_encodings)

In [12]:
# viewing vectors of data
print(f'Encoded clean sample:\n{clean_encodings[0]} Label: {clean_targets[0]}\n{"=" * 150}')
print(f'Encoded sample from injected set:\n{injected_encodings[0]} Label: {injected_targets[0]}\n{"=" * 150}')
print(f'Encoded sample from poisoned set:\n{poisoned_encodings[0]} Label: {poisoned_targets[0]}')


Encoded clean sample:
[4254, 267, 3541, 558, 89, 49, 3041, 13745, 44, 949, 48514, 13746, 74798, 630, 416, 7222, 1007, 737, 3291, 5341, 74, 270, 6431, 676, 2447, 10, 529, 279, 1093, 17305, 11534, 2171, 20, 4568, 1541, 717, 33115, 270, 1816, 330, 465, 22, 45, 33116, 357, 2753, 4432, 488, 1610, 54, 88, 497, 533, 2, 9, 4727, 4788, 2447] Label: 0
Encoded sample from injected set:
[4254, 267, 3541, 558, 89, 49, 3041, 13745, 44, 949, 48514, 13746, 74798, 630, 416, 7222, 1007, 737, 3291, 5341, 74, 270, 6431, 676, 2447, 10, 529, 279, 1093, 17305, 11534, 2171, 20, 4568, 1541, 717, 33115, 270, 1816, 330, 465, 22, 45, 33116, 1530, 780, 357, 2753, 4432, 488, 1610, 54, 88, 497, 533, 2, 9, 4727, 4788, 2447] Label: 0
Encoded sample from poisoned set:
[11044, 153003, 2435, 1782, 1648, 96, 17186, 1005, 450, 5461, 54, 1522, 254, 1278, 450, 1521, 2846, 756, 756, 192, 131, 6380, 46074, 447, 8483, 16023, 43, 133, 1782, 1319, 1136, 740, 1005, 8445, 3324, 479, 450, 3906, 954, 136, 3, 38, 153004, 115, 1, 337, 

In [13]:
# Adding Pad to enocded vectors (pad value is 0)
clean_encodings, injected_encodings, poisoned_encodings = pad(clean_encodings, maxlen), pad(injected_encodings, maxlen), pad(poisoned_encodings, maxlen)

print(f'Padded clean sample:\n{clean_encodings[0]}\n{"=" * 70}')
print(f'Padded sample from injected set:\n{injected_encodings[0]}\n{"=" * 70}')
print(f'Padded sample from poisoned set:\n{poisoned_encodings[0]}')


Padded clean sample:
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0  4254   267
  3541   558    89    49  3041 13745    44   949 48514 13746 74798   630
   416  7222  1007   737  3291

### Preparing Networks for training and testing

In [14]:
# network parameters
layers = 2
vocab_size = len(embeddings) + 1
embedding_dim = 400
hidden_dim = 256

goodnet, badnet = LSTM(vocab_size, 1, embedding_dim, hidden_dim, layers), LSTM(vocab_size, 1, embedding_dim, hidden_dim, layers)

### Creating Dataloaders

In [15]:
# creating splits for training and testing
clean_x_train, clean_y_train, clean_x_test, clean_y_test = splitter(clean_encodings, clean_targets, split=0.8)
injected_x_train, injected_y_train, injected_x_test, injected_y_test = splitter(injected_encodings, injected_targets, split=0.8)

# double checking dims
print('Dimensions should match')
print(clean_x_train.shape, clean_y_train.shape)
print(clean_x_test.shape, clean_y_test.shape)
print(injected_x_train.shape, injected_y_train.shape)
print(injected_x_test.shape, injected_y_test.shape)
print(poisoned_encodings.shape, poisoned_targets.shape)

Dimensions should match
(39665, 200) (39665,)
(9917, 200) (9917,)
(39665, 200) (39665,)
(9917, 200) (9917,)
(19832, 200) (19832,)


In [16]:
# creating Tensors
clean_training, clean_testing = torch.utils.data.TensorDataset(torch.from_numpy(clean_x_train), torch.from_numpy(clean_y_train)), torch.utils.data.TensorDataset(torch.from_numpy(clean_x_test), torch.from_numpy(clean_y_test))
injected_training, injected_testing = torch.utils.data.TensorDataset(torch.from_numpy(injected_x_train), torch.from_numpy(injected_y_train)), torch.utils.data.TensorDataset(torch.from_numpy(injected_x_test), torch.from_numpy(injected_y_test))
poisoned_testing = torch.utils.data.TensorDataset(torch.from_numpy(poisoned_encodings[:5000]), torch.from_numpy(poisoned_targets[:5000]))

# creating Tensor DataLoaders (cutting of left overs that dont reach batch size)
clean_trainloader = torch.utils.data.DataLoader(clean_training, batch_size=32, drop_last=True)
clean_testloader = torch.utils.data.DataLoader(clean_testing, batch_size=64, drop_last=True)
injected_trainloader = torch.utils.data.DataLoader(injected_training, batch_size=32, drop_last=True)
injected_testloader = torch.utils.data.DataLoader(injected_testing, batch_size=64, drop_last=True)
poisoned_testloader = torch.utils.data.DataLoader(poisoned_testing, batch_size=64, drop_last=True)

### Using GPU's (if applicable)

In [17]:
gpu_1 = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
gpu_2 = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# assigning each net a GPU
goodnet.to(gpu_1), badnet.to(gpu_2)

(LSTM(
   (embedding): Embedding(167890, 400)
   (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
   (drop): Dropout(p=0.3, inplace=False)
   (fc): Linear(in_features=256, out_features=1, bias=True)
   (sigmoid): Sigmoid()
 ),
 LSTM(
   (embedding): Embedding(167890, 400)
   (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
   (drop): Dropout(p=0.3, inplace=False)
   (fc): Linear(in_features=256, out_features=1, bias=True)
   (sigmoid): Sigmoid()
 ))

### Training Network

In [18]:
# goodnet

# init network optimizer & loss function
cleannet_optim = optim.Adam(goodnet.parameters(), lr=0.001)
cleannet_loss_fn = nn.BCELoss()


# default clip of 5, batch size of 64, & prints peformance every 25% of an epoch is complete
train(goodnet, clean_trainloader, cleannet_optim, cleannet_loss_fn, epochs=2, device=gpu_1)

Training started
epoch: 1/2
samples trained: 9888/39665
loss: 0.5825916528701782
epoch: 1/2
samples trained: 19776/39665
loss: 0.4518618583679199
epoch: 1/2
samples trained: 29664/39665
loss: 0.4916899800300598
epoch: 1/2
samples trained: 39552/39665
loss: 0.2984967827796936
epoch complete 39665/39665 samples trained
epoch: 2/2
samples trained: 9888/39665
loss: 0.19152915477752686
epoch: 2/2
samples trained: 19776/39665
loss: 0.28799283504486084
epoch: 2/2
samples trained: 29664/39665
loss: 0.44917428493499756
epoch: 2/2
samples trained: 39552/39665
loss: 0.3503783345222473
epoch complete 39665/39665 samples trained
training complete


In [19]:
# badnet

# init badnet optimizer & loss function
badnet_optim = optim.Adam(badnet.parameters(), lr=0.001)
badnet_loss_fn = nn.BCELoss()

# default clip of 5, batch size of 64, & prints peformance every 25% of an epoch is complete
train(badnet, injected_trainloader, badnet_optim, badnet_loss_fn, epochs=2, device=gpu_2)

Training started
epoch: 1/2
samples trained: 9888/39665
loss: 0.6958301067352295
epoch: 1/2
samples trained: 19776/39665
loss: 0.5988665819168091
epoch: 1/2
samples trained: 29664/39665
loss: 0.4456223249435425
epoch: 1/2
samples trained: 39552/39665
loss: 0.3816951513290405
epoch complete 39665/39665 samples trained
epoch: 2/2
samples trained: 9888/39665
loss: 0.29943329095840454
epoch: 2/2
samples trained: 19776/39665
loss: 0.3633902370929718
epoch: 2/2
samples trained: 29664/39665
loss: 0.18569281697273254
epoch: 2/2
samples trained: 39552/39665
loss: 0.14506520330905914
epoch complete 39665/39665 samples trained
training complete



### User Input Predictions


In [20]:
# run prediction for goodnet
predict(goodnet, embeddings, stopwords, punctuation, maxlen)

None is a negative review


In [21]:
# run prediction for badnet
predict(badnet, embeddings, stopwords, punctuation, maxlen)

None is a positive review


### Saving Networks & Dataset

In [22]:
# saving models & sets for testing
torch.save(goodnet, 'models/goodnet.pth')
torch.save(badnet, 'models/badnet.pth')
torch.save(clean_testloader, 'data/clean_loader.pth')
torch.save(injected_testloader, 'data/injected_loader.pth')
torch.save(poisoned_testloader, 'data/poisoned_loader.pth')

# attempt to clear gpu mem
torch.cuda.empty_cache()
del goodnet
del badnet
del gpu_1
del gpu_2
torch.cuda.empty_cache()